In [ ]:
import numpy as np

from sb3_contrib import RecurrentPPO
from stable_baselines3.common.evaluation import evaluate_policy
from sb3_contrib.common.recurrent.type_aliases import RNNStates
import gym
import torch as th
from active_critic.utils.gym_utils import make_dummy_vec_env, sample_expert_transitions_rollouts, parse_sampled_transitions, DummyExtractor


In [ ]:
from active_critic.utils.gym_utils import make_policy_dict

In [ ]:
env_tag = 'pickplace'
seq_len = 20
n_demonstrations = 1
env, vec_expert = make_dummy_vec_env(name=env_tag, seq_len=seq_len)
env, _ = make_dummy_vec_env(name=env_tag, seq_len=seq_len)
transitions, rollouts = sample_expert_transitions_rollouts(
    vec_expert.predict, env, n_demonstrations)
actions, observations, rewards = parse_sampled_transitions(transitions=transitions, extractor=DummyExtractor(), seq_len=seq_len, device='cuda')
model = RecurrentPPO("MlpLstmPolicy", env=env, verbose=1)
lstm_states = model._last_lstm_states
a = model.policy.train()

In [ ]:
obs =  th.tensor(env.reset(),device='cuda').reshape([1,-1])
obs = observations[:,0]

In [ ]:
for j in range(100000):
    rnn_states_list = [RNNStates(pi=lstm_states[0], vf=lstm_states[1])]

    for i in range(actions.shape[1]):
        _, values, log_prob, rnn_states = model.policy.forward(obs = obs, lstm_states=rnn_states_list[-1], episode_starts = th.zeros([1], device='cuda'), deterministic=False)
        rnn_states_list.append(rnn_states)

    model.policy.optimizer.zero_grad()
    total_loss = 0
    for i in range(len(rnn_states_list) - 1):
        if i == 0:
            episode_starts = th.ones([1], device='cuda')
        else:
            episode_starts = th.zeros([1], device='cuda')

        wanted_action = actions[:,i]

        value, log_prob, entropy = model.policy.evaluate_actions(obs, wanted_action, rnn_states_list[i], episode_starts = episode_starts)
        prob_true_act = th.exp(log_prob).mean()
        log_prob = log_prob.mean()
        entropy = entropy.mean()


        ent_loss = 0 * entropy
        neglogp = -log_prob
        loss = neglogp + ent_loss
        total_loss = total_loss + loss
    total_loss = total_loss.mean()
    if j%100 == 0:
        print(f'total_loss {total_loss}')
    total_loss.backward()
    model.policy.optimizer.step()


In [26]:
rnn_actions = []
rnn_states_list = [RNNStates(pi=lstm_states[0], vf=lstm_states[1])]
for i in range(actions.shape[1]):
    rnn_action, values, log_prob, rnn_states = model.policy.forward(obs = obs, lstm_states=rnn_states_list[-1], episode_starts = th.zeros([1], device='cuda'), deterministic=True)
    rnn_actions.append(rnn_action)    
    rnn_states_list.append(rnn_states)


In [28]:
from active_critic.utils.pytorch_utils import calcMSE

In [29]:
loss = 0
for i, rnn_action in enumerate(rnn_actions):
    loss += calcMSE(rnn_action, actions[:,i])

In [30]:
loss

tensor(1.6342e-05, device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:
actions

In [ ]:
o, states = model.policy.predict(obs.cpu().numpy())

In [ ]:
o

In [ ]:
actions.shape

In [ ]:
o, states = model.policy.predict(obs.cpu().numpy(), state=states)

In [ ]:
o

In [ ]:
o, states = model.policy.predict(obs.cpu().numpy(), state=states)

In [ ]:
o

In [ ]:
loss.mean().backward()

In [ ]:
rnn_states_list[-1][0][0].mean().backward()

In [ ]:
model.policy.lstm_actor.all_weights[0][0]._grad

In [ ]:
ent_weight = 1e-2

eval actions:         
obs: th.Tensor,
actions: th.Tensor,
lstm_states: RNNStates,
episode_starts: th.Tensor,

-> 
values, log_prob, distribution.entropy()

In [ ]:
_, log_prob, entropy = model.policy.evaluate_actions(obs, actions, rnn_states, episode_starts = th.ones([1], device='cuda'))
#print(f'max log: {log_prob.max()}')

prob_true_act = th.exp(log_prob).mean()
log_prob = log_prob.mean()
entropy = entropy.mean()


ent_loss = ent_weight * entropy
neglogp = -log_prob
loss = neglogp + ent_loss

In [ ]:
loss

In [ ]:
model.policy.lstm_actor.forward(obs = env.reset(), lstm_states=lstm_states, epsidode_starts = th.zeros([1]))

In [ ]:
from imitation.algorithms.bc import BC
from active_critic.utils.gym_utils import (
    make_vec_env,
    make_dummy_vec_env,
    sample_expert_transitions_rollouts,
    make_pomdp_rollouts,
    make_dummy_vec_env_pomdp,
    get_avr_succ_rew_det
)

In [ ]:
env_tag = 'pickaplce'
seq_len = 100
n_demonstrations

env, vec_expert = make_dummy_vec_env(name=env_tag, seq_len=seq_len)
val_env, _ = make_dummy_vec_env(name=env_tag, seq_len=seq_len)

transitions, rollouts = sample_expert_transitions_rollouts(
    vec_expert.predict, val_env, n_demonstrations)